This notebook is written to implement the model for Epistasis as defined in the research work titled 'Computational Framework for Statistical Epistasis Supports XOR Penetrance Function in a Living System'. 

In [23]:
# importing all the required packages
import pandas as pd
import numpy as np
import scipy.stats

In [67]:
# reading the data 
data = pd.read_csv("top10k_pruned.csv") # the last column contains the phenotype values, in this case bmi
phenotype = data.iloc[:,-1]

In [68]:
print(phenotype)

0       0.021571
1      -0.696204
2       0.745178
3       2.002737
4       0.198740
          ...   
5561    0.649946
5562    0.416354
5563   -1.510500
5564    0.646228
5565   -0.226840
Name: y, Length: 5566, dtype: float64


In [69]:
# # reading the data 
# data = pd.read_csv("BMIwTail.csv") # the last column contains the phenotype values, in this case bmi

# # storing the phenotype in the variable
# phenotype = data.iloc[:,-1]

# defining the significance level 
alpha = .05

# storing the count of number of samples in the variable n
n = data.shape[0]

# defining degrees of freedom where n is number of samples
df = n-4

# defining the critical value
critical_value = scipy.stats.t.ppf(q=1-alpha/2,df=df)


Below is the algoritm implementation for the partial correlation of interaction of genes for Epistasis. We use two different ways of encoding the interaction terms. Firstly, the traditional method of cartesian product and then by using the XOR penetrance function.

In [7]:
# Algorithm for Interaction Coefficient for Pairwise Epistasis
"""Parameters for the method:
    snp1 - First snp in the pair to be checked for interaction
    snp2 - Second snp in the pair to be checked for interaction
    phenotype - Phenotype vector
"""
def epistatis(snp1, snp2, phenotype): 
    try:
        # removing the intercept by mean centering
        snp1_tilde = snp1 - snp1.mean()
        snp2_tilde = snp2 - snp2.mean()
        phenotype_tilde = phenotype - phenotype.mean()

        # declaring the interaction vector - will contain the interaction term of snp1 and snp2
        interaction_vector = pd.Series(dtype='float64')
    
        # defining the interaction vector - either using cartesian product or XOR
        interaction_vector = (snp1%2 + snp2%2)%2 # using XOR penetrance
        #interaction_vector = snp1.mul(snp2) # using cartesian product
    
        # mean centering the interaction vector 
        interaction_vector_tilde = interaction_vector - interaction_vector.mean()
   
        # computing the dot products as explained in the algorithm
        x = (snp1_tilde.dot(interaction_vector_tilde)/snp1_tilde.dot(snp1_tilde)) * (snp1_tilde)
        u = interaction_vector_tilde - x
        y = (((interaction_vector_tilde.dot(snp2_tilde)) * (snp1_tilde.dot(snp1_tilde))) - ((snp1_tilde.dot(snp2_tilde)) * ((snp1_tilde.dot(interaction_vector_tilde))))) / ((snp2_tilde.dot(snp2_tilde)) * (snp1_tilde.dot(snp1_tilde)) - ((snp1_tilde.dot(snp2_tilde)) * (snp1_tilde.dot(snp2_tilde))))
        z = snp2_tilde - (((snp1_tilde.dot(snp2_tilde)) / (snp1_tilde.dot(snp1_tilde))) * snp1_tilde)
        v = u - (y*z)
        r = (v.dot(phenotype_tilde)) / (v.dot(v)) # interactionn coefficient
        # performing Fisher's ttest
        t_test = np.sqrt(df)*r/(np.sqrt(1-r*r))
        p_val = scipy.stats.t.sf(abs(t_test), df)
    except Exception as e:
        print("Error pair detected with error: ", e)
        r = 0
        t_test = 0
        p_val = 1

    return r, t_test, p_val # returning the interaction coefficient

In [ ]:
# applying the above defined method to the dataset

p_value_locus = [] # list of tuples containing the p_value and the two interacting loci
for i in range(0, data.shape[1]-1):
    for j in range(i+1, data.shape[1]-1):
        interacting_snp_1 = data.iloc[:,i]
        interacting_snp_2 = data.iloc[:,j]
        r, t_test, p_val = epistatis(interacting_snp_1, interacting_snp_2, phenotype)
        p_value_locus.append((p_val, data.columns[i], data.columns[j]))

        
        # printing the interacting pairs with ttest values greater than the selected critical value
        if abs(t_test) > critical_value:
            print(" Interacting SNP 1 = {0} \t  Interacting SNP 2 ={1} \t  Beta Coefficient(r) = {2}  \t t_test = {3} \t p_val = {4} ".format(data.columns[i], data.columns[j], r, t_test, p_val))

        # printing exception cases
        if r==0 and t_test==0 and p_val==1:
            print("Error Pair: ")
            print(" Interacting SNP 1 = {0} \t  Interacting SNP 2 ={1} \t  Beta Coefficient(r) = {2}  \t t_test = {3} \t p_val = {4} ".format(data.columns[i], data.columns[j], r, t_test, p_val))

In [ ]:
# performing fdr correction to correct for multiple tests

import statsmodels.stats.multitest as ssm

all_p_values = [] # list of all the p_values of the pairwise combinations
for i in range(0, len(p_value_locus)):
    all_p_values.append(p_value_locus[i][0])

accept, corrected_pvals = ssm.fdrcorrection(np.asarray(all_p_values).flatten(), alpha=0.001)

for i in range(0, len(corrected_pvals)):
    # printing the significant pairs after p value correction
    if corrected_pvals[i] < 0.001:
        print(" p-value = {0} \t adjusted p-value = {1} \t loci 1 = {2} \t loci 2 = {3} ".format( p_value_locus[i][0],  corrected_pvals[i], p_value_locus[i][1], p_value_locus[i][2]))

Algorithm implementation for 3 way epistasis detection

In [70]:
def epistatis_three_way(i, j, k, phenotype):
    try:
        snp = np.matrix(data.iloc[:,:-1])
        phenotype_tilde = phenotype - phenotype.mean()
        phenotype_tilde = np.matrix(phenotype_tilde)
        
        x = np.transpose(np.matrix(snp[:,i]))
        y = np.transpose(np.matrix(snp[:,j]))
        z = np.transpose(np.matrix(snp[:,k]))
        q = np.concatenate((x,y,z), axis=1)
        a = np.multiply(x,y)
        q = np.concatenate((q,a),axis=1)
        a = np.multiply(y,z)
        q = np.concatenate((q,a),axis=1)
        a = np.multiply(x,z)
        q = np.concatenate((q,a),axis=1)
        a = np.multiply(x,y)
        a = np.multiply(a,z)
        q = np.concatenate((q,a),axis=1)
        q = q - q.mean(axis=0)
        q = np.concatenate((q,phenotype_tilde),axis=1)
        #beta for interaction of 3way epistasis
        beta = 0
        for l in range(1,8):
          gs_curr = q[:,l]
          for m in range(l):
            gs_prev = q[:,m]
            curr = np.matmul(np.transpose(gs_curr), gs_prev)/np.matmul(np.transpose(gs_prev), gs_prev)
            if l == 7 and m == 6:
              beta = curr
            q[:,l] = q[:,l] - curr[0,0]*gs_prev
        v = q[:,6]
        resid = q[:,7]
        dot_v_v = np.matmul(np.transpose(v),v)[0,0]
        dot_resid_resid = np.matmul(np.transpose(resid),resid)[0,0]
        t_test = np.sqrt(df)*beta*np.sqrt(dot_v_v)/np.sqrt(dot_resid_resid)
        t_test = t_test[0,0]
        p_val = 2*scipy.stats.t.sf(abs(t_test), df)

    except Exception as e:
        print("Error pair detected with error: ", e)
        t_test = 0
        p_val = 1

    return t_test, p_val


In [ ]:
# applying the above defined method (3 way Epistasis) to the dataset

p_value_locus = [] # list of tuples containing the p_value and the two interacting loci
for i in range(0, data.shape[1]-1):
    for j in range(i+1, data.shape[1]-1):
        for k in range(j+1, data.shape[1]-1):
            
            t_test, p_val = epistatis_three_way(i, j, k, phenotype)
            p_value_locus.append((p_val, data.columns[i], data.columns[j], data.columns[k]))

            # print(" Interacting SNP 1 = {0} \t  Interacting SNP 2 ={1} \t Interacting SNP 3 = {2}  \t t_test = {3} \t p_val = {4} ".format(data.columns[i], data.columns[j], data.columns[k], t_test, p_val))
            # printing the interacting pairs with ttest values greater than the selected critical value
            if abs(t_test) > critical_value:
                print(" Interacting SNP 1 = {0} \t  Interacting SNP 2 ={1} \t Interacting SNP 3 = {2}  \t t_test = {3} \t p_val = {4} ".format(data.columns[i], data.columns[j], data.columns[k], t_test, p_val))

            # # printing exception cases
            # if t_test==0 and p_val==1:
            #     print("Error Pair: ")
            #     print(" Interacting SNP 1 = {0} \t  Interacting SNP 2 ={1} \t Interacting SNP 3 = {2}  \t t_test = {3} \t p_val = {4} ".format(data.columns[i], data.columns[j], data.columns[k], t_test, p_val))